In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ecephys_analyses.on_off import run_on_off_detection, get_on_off_df_filename
from ecephys_analyses.data.paths import get_datapath
import pandas as pd

from ecephys_analyses.data.channel_groups import region_depths
from ecephys_analyses.data import paths
import ecephys.io.load

from joblib import Parallel, delayed

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()

In [ ]:
condition_groups = [
    ('Allan', 'SD'),
]

data_conditions = [
    ('Allan', 'baseline-sleep-2h-circadian-match_imec1'),
    ('Allan', 'recovery-sleep-2h_imec1'),
#     ('Doppio', 'baseline-sleep-2h-circadian-match_imec1'),
#     ('Doppio', 'recovery-sleep-2h_imec1'),
]

good_only = True
pooled = True

state = 'N2'

detection_condition = 'on_off_threshold_1'

In [ ]:
def get_subregions(subject, condition, sorting_condition, region, good_only):
    
    ks_dir = paths.get_datapath(subject, condition, sorting_condition)

   # Get clusters
    if region == 'all':
        depth_interval = None
    else:
        depth_interval = region_depths[subject][condition][region]
    info_all =  ecephys.io.load.get_cluster_info(ks_dir)
    info = info_all[info_all['depth'].isin(depth_interval)]
    
    info['depth'].hist()

In [ ]:
get_subregions('Doppio', 'sleep-homeostasis-2h_imec1', 'cortex', False)

In [ ]:
get_subregionsr (
    (region, good_only, pool),
    (subject, condition),
) in itertools.product(cluster_conditions, data_conditions):
    try:
        filename = get_on_off_df_filename(region, good_only, pool) + '.csv'
        df = pd.read_csv(
            get_datapath(
                subject,
                condition,
                detection_condition,
            )/filename
        )
    except (FileNotFoundError, pd.errors.EmptyDataError):
        print(f'Cant find {subject}, {condition}, {region}: pass')
        continue
    df['subject'] = subject
    df['condition'] = condition
    df['region'] = region
    df['good_only'] = good_only
    df['pool'] = pool
    dfs.append(df)

data_df = pd.concat(dfs)

In [ ]:
alt.Chart(data_df[0:100]).transform_density(
    'duration',
    as_=['duration', 'density'],
).mark_area().encode(
    x="duration:Q",
    y='density:Q',
)

In [ ]:
chart = alt.Chart(data_df).transform_density(
    'duration',
    as_=['duration', 'density'],
    groupby=['subject', 'condition', 'region', 'pool', 'good_only'],
).mark_area(
    opacity=0.5,
    interpolate='step',
).encode(
    x = alt.X('duration:Q',
#               axis=alt.Axis(title='Off duration (s)'),
#               scale=alt.Scale(zero=True),
#               bin=alt.Bin(maxbins=50),
    ),
    y = alt.Y('density:Q', stack=None),
    color=alt.Color('condition:N'),
    row='subject',
)

alt.ConcatChart(
    concat=[
      chart.transform_filter(
          alt.datum.region == region
      ).properties(title=region)
      for region in sorted(data_df.region.unique())
#       for region in ['cortex']
    ],
#     columns=4
).configure_title(
    fontSize=20,
#     font='Courier',
#     anchor='middle',
#     color='gray',
#     align='left'
).resolve_axis(
    x='independent',
    y='independent'
).resolve_scale(
    x='independent', 
    y='independent'
)

In [ ]:
alt.Chart(data_df[0:1000]).transform_density(
    'duration',
    as_=['duration', 'density'],
    groupby=['subject', 'region', 'pool', 'good_only']
).mark_area(
    opacity=0.3,
    interpolate='step',
).encode(
    alt.X('duration:Q', bin=alt.Bin(maxbins=50)),
    alt.Y('density:Q', stack=None),
    alt.Color('condition:N'),
    row='subject',
    column='region',
).resolve_scale(
    y='independent',
    x='independent',
)

In [ ]:
data_df

In [ ]:
# data_df[(data_df['region'] == 'cortex_2') & data_df['start_time'].isin((30, 60))]
pd.set_option('display.max_rows', None)

data_df[(data_df['region'] == 'cortex_2')]